CSCI 3832, Spring 2020, Lecture 34 — HW 5 examples & getting started

In [4]:
import numpy as np
from collections import Counter
from scipy.special import softmax


Section 1: building a Mini-HMM
========================

In [5]:
class GreedyHMM:
  
    def __init__(self):
        # here are the items that we'll need to gather
        self.pi = Counter() # pi
        self.transitions = {} # this is A
        self.emissions = {} # this is B
        self.states = set() # Q
        self.vocab = set() # O

    def train(self, example):
        """
        Trains this model based on the given input data
        params: examples - a list of (token, label) tuples
        return: None
        This example is going to be list of token, label, in
        hw5 it is a list of lists of examples, this is one example
        We won't be implementing laplace smoothing which is something you
        need to do in hw5
        """
        
        # pi (initial probability) * p(w_i | t_i)
        
        # prev_prob * p(w_i | t_i) * p(t_i | t_i-1)
        
        # in hw5 there will be another loop through the sentences
        for i in range(len(example)):
            # tuple of word and state so need 0 word and 1 state
            word = example[i][0]
            state = example[i][1]
            if i == 0:
                # count(sentences that start with state) / count of sentences
                # the fraction of sentences that start with our state
                # how many sentences start with our state
                self.pi[state] += 1
            else:
                # p(t_i | t_(i-1))
                # count(t_i - 1, t_i) / count(t_i - 1)
                # transitions[prev_state][state] = count
                # sum(transitions[prev_state].values()) = count(t_i - 1)
                if example[i - 1][1] not in self.transitions:
                    self.transitions[example[i - 1][1]] = Counter()
                # nested dictionary, nicer, could do seperate dictionaries
                self.transitions[example[i - 1][1]][state] += 1
                
                #TODO: think about what happens for the final state in the sentence
        
            # p(w_i | t_i)
            # count(t_i, w_i) / count(t_i)
            # emissions[prev_state][word] = count(t_i, w_i)
            # now we can sum are emissions just like we did with transitions
            # sum(emissions[state].values()) = count(t_i)
            if state not in self.emissions:
                self.emissions[state] = Counter()
            self.emissions[state][word] += 1
            
        
        #TODO: make into percentages 
        print(self.pi)
        
        print(self.transitions)
        print(self.emissions)
        pass
    
    def greedy_decode(self, data):
        """
        params: data - a list of tokens
        return: a list of [(token, label)...] tuples
        We will be doing viterbi with beam width of 1
        """
        
        # hw5 we will run viterbi
        # here we are going to implement it with a beam size of 1
        prev_prob = 0
        prev_state = None
        for i in range(len(data)):
            word = data[i]
            max_prob = 0
            max_state = None
            # could save all the states in a seperate variable if you wanted
            for state in self.emissions.keys():
                emissions = self.emissions[state][word] / sum(self.emissions[state].values())
                if i == 0:
                    # pi * emission prob
                    pi_val = self.pi[state] / sum(self.pi.values())
                    prob = emission * pi_val
                    # probabilities we are beginning in each state
                else:
                    transition = self.transitions[prev_state][state] / sum(self.transitinons[prev_state].values())
                    prob = prev_prob * transition * emission
                # could also do a list and argmax
                if max_state in None or prob > max_prob:
                    max_state = state
                    max_prob = prob
            
            prev_prob = max_prob
            prev_state = max_state
            print(word)
            print(max_prob)
            print(max_state)
            # if you wanted this as a list you could concatenate the results and return an entire list
        
        pass
            

In [6]:

train_data = [("I", "NOUN"), ("went", "OTHER"), ("to", "OTHER"), ("the", "OTHER"), ("zoo", "NOUN"), ("and", "OTHER"), \
               ("I", "NOUN"), ("saw", "OTHER"), ("a", "OTHER"), ("penguin", "NOUN"), ("and", "OTHER"), ("sparkly", "OTHER"), ("jaguars", "NOUN"), ("!", "OTHER")]
test_data = "I went to the penguin park .".split()


In [7]:
hmm = GreedyHMM()
hmm.train(train_data)
hmm.greedy_decode(test_data)

Counter({'NOUN': 1})
{'NOUN': Counter({'OTHER': 5}), 'OTHER': Counter({'OTHER': 4, 'NOUN': 4})}
{'NOUN': Counter({'I': 2, 'zoo': 1, 'penguin': 1, 'jaguars': 1}), 'OTHER': Counter({'and': 2, 'went': 1, 'to': 1, 'the': 1, 'saw': 1, 'a': 1, 'sparkly': 1, '!': 1})}


Differences between this HMM and the one that you're implementing for HW 5?


Need to handle multiple examples

Need to laplace smooth

Do all of viterbi

Section 2: building a Mini-MEMM
========================

In [8]:
class GreedyMEMM:
    # this can get very slow very quickly
  
    def __init__(self):
        # TODO: randomly initialize your weights
        self.weights = np.array([[1, 0, 1],[2, -3, -0.5],[0, 0.5, -1]])
        #self.weights = 2 * np.random.random_sample((len(self.categories), self.num_feats + 1)) - 1
        self.num_feats = 2
        self.states = [1, 2, 3]

    def train(self, examples, iterations = 100, learning_rate = 0.1):
        """
        Trains this model based on the given input data
        params: examples - a list of (features, label) data, one per example
        return: None
        """
        
        # running SGD
        
        # while we still need to iterate
        # loop over our examples (in random order)
            # get the feature representation
            # featurized = np.array(example[0] + [1]) # add bias in
            # calculate y_hat
            # z = np.dot(featurized, self.weights.T)
            # y_hat = softmax(z)
            
            # calculate gradients
            # - (1 * {y - k} - p(y - k | x)) (then * x)
            '''
            true_label = example[1]
            indicators = [1 if j == true_label else 0 for j in self.states]
            gradient_factors = -(1 * indicators - y_hat)
            gradients = gradient_factors.reshapte(gradient_factors.shape[0], 1) @ featurized.reshape(featurized.shape[0])
            '''
            
            # update your weights according to the gradients multiplied by the learning rate
            
        
        pass
    
    def classify_single(self, data):
        """
        params: data - a list of features
        return: a list of [(token, label)...] tuples
        """
        pass

In [9]:
# these are our two features and then a label
train_data = [([3, 0.75], 1), ([0, 1], 2), ([1, 1], 3), ([2, 4], 2)]
test_data = [2, 4]

In [10]:
memm = GreedyMEMM()
memm.train(train_data, iterations = 100)
memm.classify_single(test_data)

Differences between this MEMM and the one that you're implementing for HW 5?

General questions:
1. what happens with unknown words?
    1. for the HMM?
    2. for the MEMM?
    
We are not going to worry about it


HW 5 format questions:
1. What is the `generate_probabilities` function doing?
    1. What do the return values represent?
2. What is the `decode` function doing?
3. How to use the `precision`, `recall`, `f1` functions?